In [1]:
!nvidia-smi

Thu Jan 26 13:36:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.7 MB/s eta 0:00:00


In [39]:
import pandas as pd

import tensorflow as tf

import keras

import torch

from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertForNextSentencePrediction, pipeline, BertForQuestionAnswering, BertForTokenClassification, BertForSequenceClassification

from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sb

In [ ]:
#email = pd.read_csv('../../../../../../data/r4.2/email.csv')
file = pd.read_csv('/content/drive/MyDrive/Praxis/data/file.csv')
#http = pd.read_csv('../../../../../../data/r4.2/http.csv')
#logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
#device = pd.read_csv('../../../../../../data/r4.2/device.csv')
#tweets = pd.read_csv('/content/drive/MyDrive/Praxis/code/tweets.csv')
#enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')


In [ ]:
#files = [file,email,http,logon,device]

In [ ]:
#for x in files:
#    print(x.shape)
file.shape

(445581, 6)

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
named_params = list(model.named_parameters())

In [40]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer.encode_plus('Ken is the best!')

{'input_ids': [101, 6358, 2003, 1996, 2190, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
response = model(torch.tensor(tokenizer.encode('Ken is the best')).unsqueeze(0))

In [ ]:
#response

In [ ]:
response.pooler_output.shape

torch.Size([1, 768])

In [ ]:
model.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [ ]:
cls_embedding = response.last_hidden_state[:, 0, :].unsqueeze(0)

In [ ]:
cls_embedding.shape

torch.Size([1, 1, 768])

In [ ]:
model.pooler(cls_embedding).shape

torch.Size([1, 768])

In [ ]:
(model.pooler(cls_embedding) == response.pooler_output).all()

tensor(True)

In [ ]:
'sinan' in tokenizer.vocab

False

In [ ]:
i=0

In [ ]:
encoded_file_content = tokenizer.encode_plus(file['content'][0])

In [ ]:
encoded_file_content

{'input_ids': [101, 1040, 2692, 1011, 12935, 1011, 2340, 1011, 1041, 2692, 1011, 17350, 1011, 29491, 1011, 20720, 1011, 1041, 2487, 2076, 7669, 3452, 17138, 3904, 9048, 16173, 2102, 4711, 6048, 4105, 2011, 2738, 2104, 3356, 2520, 2019, 5955, 2261, 9576, 5590, 5002, 2153, 19817, 25438, 18083, 2071, 8003, 2205, 4415, 2309, 4672, 2185, 2219, 8489, 2068, 5941, 6470, 2114, 2035, 2646, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#model.embeddings

In [ ]:
example = "Ken is the best guy!"

In [ ]:
tokenizer.encode(example, return_tensors='pt')

tensor([[ 101, 6358, 2003, 1996, 2190, 3124,  999,  102]])

In [ ]:
model.embeddings.word_embeddings(tokenizer.encode(example, return_tensors='pt'))

tensor([[[ 0.0136, -0.0265, -0.0235,  ...,  0.0087,  0.0071,  0.0151],
         [-0.0098, -0.0581, -0.0190,  ..., -0.0698, -0.0208, -0.0665],
         [-0.0360, -0.0246, -0.0257,  ...,  0.0034, -0.0018,  0.0269],
         ...,
         [-0.0141, -0.1070,  0.0017,  ..., -0.0354, -0.0362, -0.0096],
         [ 0.0298, -0.0373, -0.0356,  ...,  0.0161,  0.0192,  0.0173],
         [-0.0145, -0.0100,  0.0060,  ..., -0.0250,  0.0046, -0.0015]]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
model.embeddings.position_embeddings

Embedding(512, 768)

In [ ]:
model.embeddings.position_embeddings(torch.LongTensor(range(9)))

tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [ 6.4257e-03, -1.7664e-02, -2.2067e-02,  ...,  6.7531e-04,
          1.1108e-02,  3.7521e-03],
        [ 6.2613e-04, -1.6089e-02, -7.6365e-03,  ...,  5.3390e-03,
          1.5909e-02,  1.8119e-03],
        [ 2.0350e-03, -1.4569e-02, -1.4560e-02,  ...,  1.4465e-02,
          1.0401e-02,  3.7587e-03]], grad_fn=<EmbeddingBackward0>)

In [ ]:
torch.LongTensor([0]*6)

tensor([0, 0, 0, 0, 0, 0])

In [ ]:
model.embeddings.token_type_embeddings(torch.LongTensor([0]*6))

tensor([[ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
model.embeddings(tokenizer.encode(example, return_tensors='pt'))

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3742, -0.3580, -0.1415,  ..., -0.3706,  0.5141, -0.9572],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [-0.0221, -1.6964,  0.0810,  ..., -0.2988, -0.0200,  0.0699],
         [ 0.8539, -0.5522, -0.6580,  ...,  0.3174,  0.6846,  0.4836],
         [-0.2551, -0.2671,  0.0799,  ..., -0.4287,  0.3415, -0.1055]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
model.embeddings.LayerNorm(
    model.embeddings.word_embeddings(tokenizer.encode(example, return_tensors='pt')) +
    model.embeddings.position_embeddings(torch.LongTensor(range(8))) +
    model.embeddings.token_type_embeddings(torch.LongTensor([0]*8))
)

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3742, -0.3580, -0.1415,  ..., -0.3706,  0.5141, -0.9572],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [-0.0221, -1.6964,  0.0810,  ..., -0.2988, -0.0200,  0.0699],
         [ 0.8539, -0.5522, -0.6580,  ...,  0.3174,  0.6846,  0.4836],
         [-0.2551, -0.2671,  0.0799,  ..., -0.4287,  0.3415, -0.1055]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
model.embeddings(tokenizer.encode(example, return_tensors='pt')).size()

torch.Size([1, 8, 768])

In [ ]:
bert_lm = BertForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#bert_lm

In [ ]:
nlp = pipeline('fill-mask', model = 'bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
type(nlp.model)

transformers.models.bert.modeling_bert.BertForMaskedLM

In [ ]:
nlp.tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
preds = nlp(f"If you dont {nlp.tokenizer.mask_token} at the sign, you will get a ticket.")

In [ ]:
for p in preds:
  print(f"token: {p['token_str']}")

token: look
token: stop
token: glance
token: turn
token: stare


In [ ]:
bert_nsp = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#bert_nsp

In [ ]:
text1 = "Ken is the man."
text2 = "He is the best as everything."

In [ ]:
inputs = tokenizer(text1, text2, return_tensors='pt' )

In [ ]:
inputs

{'input_ids': tensor([[ 101, 6358, 2003, 1996, 2158, 1012,  102, 2002, 2003, 1996, 2190, 2004,
         2673, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = bert_nsp(**inputs)

In [ ]:
outputs

NextSentencePredictorOutput(loss=None, logits=tensor([[ 6.0411, -5.5872]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
outputs = bert_nsp(**inputs, labels=torch.LongTensor([0]))

In [ ]:
outputs

NextSentencePredictorOutput(loss=tensor(8.9407e-06, grad_fn=<NllLossBackward0>), logits=tensor([[ 6.0411, -5.5872]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
outputs = bert_nsp(**inputs, labels=torch.LongTensor([1]))

In [ ]:
outputs

NextSentencePredictorOutput(loss=tensor(11.6282, grad_fn=<NllLossBackward0>), logits=tensor([[ 6.0411, -5.5872]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
bert_sq = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
#bert_sq

In [ ]:
bert_sq.classifier

Linear(in_features=768, out_features=3, bias=True)

In [ ]:
finbert = pipeline(task="text-classification", model='ProsusAI/finbert', tokenizer='ProsusAI/finbert')

In [ ]:
finbert("The year was 1929.")

[{'label': 'neutral', 'score': 0.8950798511505127}]

In [ ]:
ner_example = pipeline(task='ner', model='bert-base-cased', tokenizer='bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
#ner_example("My name is Ken and I am from Miami")

In [ ]:
bert_qa = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
#bert_qa

In [ ]:
model_qa_squad2 = "deepset/roberta-base-squad2"

In [ ]:
squad_qa = pipeline(task="question-answering", model=model_qa_squad2, tokenizer=model_qa_squad2)

In [ ]:
sequence = "Where does Ken work?", "Ken is working for PowerSchool and John works for the FBI."

In [ ]:
squad_qa(*sequence)

{'score': 0.8899106383323669, 'start': 19, 'end': 30, 'answer': 'PowerSchool'}

In [ ]:
nlp = pipeline(task='fill-mask', model='distilbert-base-cased')

In [ ]:
print(type(nlp.model))

<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM'>


In [ ]:
preds = nlp(f"if you don't {nlp.tokenizer.mask_token} at the sign, you will get a ticket")

In [ ]:
for p in preds:
  print(f"token: {p['token_str']}, score: {100*p['score']:,.2f}%")

token: look, score: 58.75%
token: stop, score: 7.81%
token: glance, score: 4.35%
token: arrive, score: 2.23%
token: appear, score: 1.89%


We can average the hidden layer outputs to make our own vector instead of the actual pooler that comes with pytorch class.

In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding 

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
from datasets import load_metric, Dataset

In [3]:
snips_file = pd.read_csv('/content/drive/MyDrive/Praxis/data/train.csv')

In [4]:
snips_file

,text,slot,intent
0,listen to westbam alumb allergic on google music,O O B-artist O B-album O B-service I-service,PlayMusic
1,add step to me to the 50 clásicos playlist,O B-entity_name I-entity_name I-entity_name O ...,AddToPlaylist
2,i give this current textbook a rating value of...,O O O B-object_select B-object_type O O O O B-...,RateBook
3,play the song little robin redbreast,O O B-music_item B-track I-track I-track,PlayMusic
4,please add iris dement to my playlist this is ...,O O B-artist I-artist O B-playlist_owner O B-p...,AddToPlaylist
...,...,...,...
13079,i want to eat choucroute at a brasserie for 8,O O O O B-served_dish O O B-restaurant_type O ...,BookRestaurant
13080,play funky heavy bluesy,O B-playlist I-playlist I-playlist,PlayMusic
13081,rate the current album 2 points out of 6,O O B-object_select B-object_type B-rating_val...,RateBook
13082,go to the photograph the inflated tear,O O O B-object_type B-object_name I-object_nam...,SearchCreativeWork


In [ ]:
snips_file['intent'].size

13084

In [5]:
snips_file['slot'][0]

'O O B-artist O B-album O B-service I-service'

In [6]:
sequence_labels_raw = [list(snips_file['slot'])]

In [8]:
sequence_labels_raw[0]

['O O B-artist O B-album O B-service I-service',
 'O B-entity_name I-entity_name I-entity_name O O B-playlist I-playlist O',
 'O O O B-object_select B-object_type O O O O B-rating_value O O O O O B-best_rating',
 'O O B-music_item B-track I-track I-track',
 'O O B-artist I-artist O B-playlist_owner O B-playlist I-playlist I-playlist',
 'O B-artist I-artist I-artist O B-playlist_owner B-playlist I-playlist I-playlist O',
 'O O O O O B-year O',
 'O O B-sort B-music_item O B-artist I-artist',
 'O B-movie_name I-movie_name',
 'O O O O B-party_size_number O B-state',
 'O O O O O B-city B-state O B-timeRange I-timeRange I-timeRange I-timeRange I-timeRange I-timeRange I-timeRange',
 'O B-object_select B-object_part_of_series_type O B-rating_value',
 'O O O O O B-object_type I-object_type O B-movie_type B-spatial_relation I-spatial_relation I-spatial_relation',
 'O O O O O B-year O O B-artist',
 'O O O O O O O O B-geographic_poi I-geographic_poi I-geographic_poi B-spatial_relation B-timeRange 

In [11]:
sequence_labels = []

def Convert(string):
  li = list(string.split(" "))
  return li

for sequence_row in sequence_labels_raw[0]:
  temp = Convert(sequence_row)
  for word in temp:
    if word not in sequence_labels:
      sequence_labels.append(word) 

In [12]:
sequence_labels

['O',
 'B-artist',
 'B-album',
 'B-service',
 'I-service',
 'B-entity_name',
 'I-entity_name',
 'B-playlist',
 'I-playlist',
 'B-object_select',
 'B-object_type',
 'B-rating_value',
 'B-best_rating',
 'B-music_item',
 'B-track',
 'I-track',
 'I-artist',
 'B-playlist_owner',
 'B-year',
 'B-sort',
 'B-movie_name',
 'I-movie_name',
 'B-party_size_number',
 'B-state',
 'B-city',
 'B-timeRange',
 'I-timeRange',
 'B-object_part_of_series_type',
 'I-object_type',
 'B-movie_type',
 'B-spatial_relation',
 'I-spatial_relation',
 'B-geographic_poi',
 'I-geographic_poi',
 'B-restaurant_type',
 'I-city',
 'B-party_size_description',
 'I-party_size_description',
 'B-object_location_type',
 'I-object_location_type',
 'B-object_name',
 'I-object_name',
 'I-movie_type',
 'B-rating_unit',
 'I-sort',
 'B-location_name',
 'I-location_name',
 'B-current_location',
 'I-current_location',
 'I-playlist_owner',
 'B-served_dish',
 'B-country',
 'B-condition_temperature',
 'B-poi',
 'I-poi',
 'B-condition_descri

In [13]:
intents_raw = list(snips_file['intent'])

In [14]:
intents_raw

['PlayMusic',
 'AddToPlaylist',
 'RateBook',
 'PlayMusic',
 'AddToPlaylist',
 'AddToPlaylist',
 'PlayMusic',
 'PlayMusic',
 'SearchScreeningEvent',
 'BookRestaurant',
 'GetWeather',
 'RateBook',
 'SearchScreeningEvent',
 'PlayMusic',
 'GetWeather',
 'BookRestaurant',
 'BookRestaurant',
 'AddToPlaylist',
 'BookRestaurant',
 'AddToPlaylist',
 'SearchScreeningEvent',
 'AddToPlaylist',
 'SearchCreativeWork',
 'SearchCreativeWork',
 'AddToPlaylist',
 'SearchScreeningEvent',
 'SearchCreativeWork',
 'RateBook',
 'SearchScreeningEvent',
 'SearchScreeningEvent',
 'PlayMusic',
 'RateBook',
 'AddToPlaylist',
 'SearchScreeningEvent',
 'SearchCreativeWork',
 'BookRestaurant',
 'GetWeather',
 'PlayMusic',
 'AddToPlaylist',
 'SearchCreativeWork',
 'SearchCreativeWork',
 'BookRestaurant',
 'SearchScreeningEvent',
 'PlayMusic',
 'GetWeather',
 'AddToPlaylist',
 'GetWeather',
 'RateBook',
 'SearchScreeningEvent',
 'SearchScreeningEvent',
 'BookRestaurant',
 'PlayMusic',
 'PlayMusic',
 'BookRestaurant',


In [15]:
intents = [*set(list(intents_raw))]

In [16]:
intents

['PlayMusic',
 'BookRestaurant',
 'SearchScreeningEvent',
 'RateBook',
 'SearchCreativeWork',
 'AddToPlaylist',
 'GetWeather']

In [31]:
utterances_raw = list(snips_file['text'])

In [33]:
utterances_raw

['listen to westbam alumb allergic on google music',
 'add step to me to the 50 clásicos playlist',
 'i give this current textbook a rating value of 1 and a best rating of 6',
 'play the song little robin redbreast',
 'please add iris dement to my playlist this is selena',
 'add slimm cutta calhoun to my this is prince playlist',
 'i want to listen to seventies music',
 'play a popular chant by brian epstein',
 'find fish story',
 'book a spot for 3 in mt',
 'i need a forecast for jetmore massachusetts in 1 hour and 1 second from now',
 'rate this series a 5',
 'give me a list of movie times for films in the area',
 'can you play me some eighties music by adele',
 'please let me know the weather forcast of stanislaus national forest far in nine months',
 'book a restaurant for eight people in six years',
 'i need to book a restaurant in fork mountain sc for valarie mari and i',
 'add to playlist confidence boost here comes santa claus',
 'book a restaurant at sixteen o clock in sc',
 '

In [34]:
utterances = []
for j in utterances_raw:
  k = j.split(" ")
  utterances.append(k)

In [36]:
utterances[0]

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']

In [41]:
token_seq = tokenizer.encode('utterances[0]')

In [42]:
token_seq

[101, 14395, 26755, 1031, 1014, 1033, 102]